This notebook will create the "gen_charac.csv" table used by grid2op.

# 1) read the .m file and save it into pandapower format

In [1]:
import os
import pandapower as pp
from matpowercaseframes import CaseFrames
import numpy as np
import pandas as pd

In [2]:

path = "h20200101_0000-m"
fn_ = "h20200101_0000_nohvdc_noxnull.m"
fn_ = "h20200101_0100_nobreaker_nostorage_nohvdc.m"
fn_ref = "h20200101_0100_nobreaker_nostorage_nohvdc_va_ac_ref.csv"
grid = pp.converter.from_mpc(os.path.join(path, fn_))
pp.rundcpp(grid)
# now check if voltage angles match
va_ref = pd.read_csv(os.path.join(path, fn_ref))
pp.runpp(grid)
grid.res_bus

,vm_pu,va_degree,p_mw,q_mvar
0,1.046800,-3.549403,1.343116,1.819121
1,1.046700,-3.694747,3.659982,4.537707
2,1.006933,1.229403,55.856193,37.000000
3,1.014355,-3.584434,22.963101,15.000000
4,1.043385,-4.129789,22.032165,14.000000
...,...,...,...,...
68,1.050000,19.369438,0.000000,-11.005147
69,1.050000,21.877443,-64.806000,32.512540
70,1.050000,14.850281,0.000000,-40.131782
71,1.036771,24.551971,0.000000,0.000000


In [3]:
va_ref

,id,v_mag
0,VL-101_0,1.05
1,VL-102_0,1.05
2,VL-103_0,1.03
3,VL-104_0,1.04
4,VL-105_0,1.05
...,...,...
68,VL-321_0,1.22
69,VL-322_0,1.22
70,VL-323_0,1.25
71,VL-324_0,1.27


In [4]:
# save it to json
pp.rundcpp(grid)
pp.to_json(grid, "grid_v1.json")

## tentative debug du reseau

In [5]:
# res = pp.converter.to_mpc(grid, os.path.join(path, "converted.m"))
# pp.to_excel(grid, os.path.join(path, "converted_pp.xlsx"))

In [6]:
# orig = CaseFrames(os.path.join(path, fn_))
# orig.to_excel(os.path.join(path, "orig.xlsx"))

In [7]:
if False:
    convert = np.sort(np.unique(orig.branch.iloc[:,[0,1]].values.flatten()))
    for (o_id, o_el), (c_oid, c_exid, c_r, c_x, c_b) in zip(orig.branch.iterrows(),
                                                        zip(res["mpc"]["branch"][:,0] - 1,  # python 0 based, matlab 1 based
                                                            res["mpc"]["branch"][:,1] - 1,  # python 0 based, matlab 1 based
                                                            res["mpc"]["branch"][:,2],
                                                            res["mpc"]["branch"][:,3],
                                                            res["mpc"]["branch"][:,4],
                                                            )):
        assert o_el[0] == convert[int(c_oid)], f"error for {o_id} for origin bus"
        assert o_el[1] == convert[int(c_exid)], f"error for {o_id} for ex bus"
        assert np.allclose(o_el[2], c_r), f"error for {o_id} for r"
        assert np.allclose(o_el[3], c_x), f"error for {o_id} for x"
        assert np.allclose(o_el[4], c_b), f"error for {o_id} for b"

# 2) Modify the grid to be "easier to manipulate"

In [8]:
grid = pp.from_json("grid_v1.json")

In [9]:
# handle the slack in the generator, and remove the "ext_grid", 
# otherwise pandapower might behave not as expected
for id_, el in grid.ext_grid.iterrows():
    pp.create_gen(grid,
                  bus=el["bus"],
                  slack=True,
                  vm_pu=el["vm_pu"],
                  slack_weight=el["slack_weight"],
                  p_mw=0.,
                  in_service=el["in_service"],
                  min_p_mw=el["min_p_mw"],
                  max_p_mw=el["max_p_mw"],
                  min_q_mvar=el["min_q_mvar"],
                  max_q_mvar=el["max_q_mvar"])
# remove the now useless tables
grid.ext_grid.drop(grid.ext_grid.index, inplace=True)

First, let's move the static generator to sandard generators (we will use DC so there is no issue with that !)

In [10]:
# copy sgen into generators
for id_, el in grid.sgen.iterrows():
    # fix voltages (sgen might be connected to buses where there are buses)
    vm_pu = 1.0
    has_already_gen = grid.gen.bus == el["bus"]
    if has_already_gen.any():
        vm_pu = grid.gen.loc[has_already_gen]["vm_pu"].values[0]
        
    # now add the generator as a standard generator
    pp.create_gen(grid,
                  bus=el["bus"],
                  vm_pu=vm_pu,
                  p_mw=el["p_mw"],
                  in_service=el["in_service"],
                  min_p_mw=el["min_p_mw"],
                  max_p_mw=el["max_p_mw"],
                  min_q_mvar=el["min_q_mvar"],
                  max_q_mvar=el["max_q_mvar"])
# remove the now useless tables
grid.sgen.drop(grid.sgen.index, inplace=True)
grid.res_sgen.drop(grid.res_sgen.index, inplace=True)

In [11]:
# check that the sum of generation and load matches (hopefully)
grid.load["p_mw"].sum() - grid.gen["p_mw"].sum()

56.18951851999964

In [12]:
pp.rundcpp(grid)

In [13]:
pp.to_json(grid, "grid_final.json")
pp.to_json(grid, "grid.json")